## Load Validation Data

In [1]:
import pandas as pd

df = pd.read_pickle("../../eval_examples.pkl")
df

,input,label,prediction
0,<s>item<extra_id_5><extra_id_2><extra_id_5>kwa...,str,str
1,<s>response<extra_id_5><extra_id_2><extra_id_5...,object,object
2,<s>config<extra_id_5><extra_id_2><extra_id_5>=...,str,str
3,<s>unique_id<extra_id_5><extra_id_2><extra_id_...,str,None
4,<s>repr<extra_id_5><extra_id_2><extra_id_5>mod...,callable,callable
...,...,...,...
10295,<s>self<extra_id_5><extra_id_2><extra_id_5>)\n...,object,object
10296,<s>split<extra_id_5><extra_id_4><extra_id_5> n...,callable,callable
10297,<s>name<extra_id_5><extra_id_4><extra_id_5>=Fa...,str,str
10298,<s>_guess_distribution<extra_id_5><extra_id_3>...,dict,dict


## Overall Accuracy

In [2]:
def accuracy(df):
    return len(df[df["label"] == df["prediction"]]) / len(df)

accuracy(df)

0.8807766990291263

## Per-kind Accuracy (name vs attribute vs call)

In [3]:
name_df = df[df["input"].str.contains("<extra_id_5><extra_id_2>")]
print(f"Name     : {accuracy(name_df)}")
attribute_df = df[df["input"].str.contains("<extra_id_5><extra_id_4>")]
print(f"Attribute: {accuracy(attribute_df)}")
call_df = df[df["input"].str.contains("<extra_id_5><extra_id_3>")]
print(f"Call     : {accuracy(call_df)}")

Name     : 0.8667531204409142
Attribute: 0.9088050314465409
Call     : 0.8934383202099737


## Per-category Accuracy (i.e., comparing different abstracted values)

In [4]:
all_categories = set(df["label"])

print(f"{len(all_categories)} categories")

def accuracy_per_category(df, category):
    sub_df = df[df["label"] == category]
    return accuracy(sub_df)

def most_common_mispredictions(df):
    mispredictions = df[df["label"] != df["prediction"]]["prediction"]
    all = mispredictions.value_counts()
    top = all.head(5)
    s = ""
    for i in range(len(top)):
        perc = round(top[i]/len(mispredictions), 2)
        s += f"{top.index[i]}={perc}, "
    return s

pd.options.display.max_colwidth = 100
raw_category_accuracies = [[c, accuracy_per_category(df,c), len(df[df["label"] == c]), most_common_mispredictions(df[df["label"] == c])] for c in all_categories]
category_accuracies = pd.DataFrame(raw_category_accuracies, columns=["category", "accuracy", "count", "mispredictions"])
category_accuracies.sort_values(by="accuracy", ascending=False)
    


12 categories


,category,accuracy,count,mispredictions
5,callable,0.975660,2917,"str=0.28, object=0.24, resource=0.14, None=0.13, int=0.08,"
2,object,0.920606,3300,"None=0.3, str=0.29, list=0.14, dict=0.13, int=0.05,"
11,bool,0.884527,433,"int=0.26, str=0.22, None=0.2, list=0.14, object=0.1,"
9,dict,0.863248,468,"None=0.3, list=0.25, str=0.17, int=0.11, object=0.11,"
6,str,0.839142,1119,"None=0.42, int=0.18, object=0.15, list=0.12, bool=0.04,"
0,list,0.825911,494,"None=0.23, str=0.2, int=0.14, dict=0.14, object=0.12,"
10,set,0.776316,76,"list=0.47, None=0.18, object=0.12, dict=0.12, int=0.06,"
8,int,0.775362,414,"str=0.42, None=0.27, list=0.08, float=0.08, bool=0.06,"
3,resource,0.739130,92,"callable=0.46, None=0.17, str=0.12, dict=0.08, list=0.04,"
7,tuple,0.660550,218,"list=0.46, None=0.19, str=0.12, dict=0.07, int=0.05,"


## Random sample of mispredicted examples

In [5]:
mispredictions = df[df["label"] != df["prediction"]]
for _ in range(10):
#     example = mispredictions.sample()
#     print(f"================================\n")
#     print(f"Input:\n{example['input'].iloc(2)}")
    example = mispredictions.sample()
    input = example["input"].iloc[0]
    label = example["label"].iloc[0]
    print(f"================================\n")
    print(f"{input}\n\n-->{label}")




<s>get_formsets_with_inlines<extra_id_5><extra_id_3><extra_id_5>label,
                "admin/object_history.html",
            ],
            context,
        )

    def get_formset_kwargs(self, request, obj, inline, prefix):
        formset_params = {
            "instance": obj,
            "prefix": prefix,
            "queryset": inline.get_queryset(request),
        }
        if request.method == "POST":
            formset_params.update(
                {
                    "data": request.POST.copy(),
                    "files": request.FILES,
                    "save_as_new": "_saveasnew" in request.POST,
                }
            )
        return formset_params

    def _create_formsets(self, request, obj, change):
        "Helper function to generate formsets for add/change_view."
        formsets = []
        inline_instances = []
        prefixes = {}
        get_formsets_args = [request]
        if change:
            get_formsets_args.append(obj)
        for Form